Download the List_of_postal_codes_of_Canada to an html file:

In [3]:
!curl https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M > postal_codes.html
print('Data downloaded!')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 78670  100 78670    0     0  44122      0  0:00:01  0:00:01 --:--:-- 44122
Data downloaded!


parse the html file using BeautifulSoup:

In [15]:
from bs4 import BeautifulSoup

with open('postal_codes.html') as html_file:
    html_data = html_file.read()

    soup=BeautifulSoup(html_data,"lxml")
    #print(soup.prettify())
    result = soup.find("table")
    

<class 'str'>


In [73]:
str(result)[0:1000]

'<table class="wikitable sortable">\n<tbody><tr>\n<th>Postcode</th>\n<th>Borough</th>\n<th>Neighbourhood\n</th></tr>\n<tr>\n<td>M1A</td>\n<td>Not assigned</td>\n<td>Not assigned\n</td></tr>\n<tr>\n<td>M2A</td>\n<td>Not assigned</td>\n<td>Not assigned\n</td></tr>\n<tr>\n<td>M3A</td>\n<td><a href="/wiki/North_York" title="North York">North York</a></td>\n<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>\n</td></tr>\n<tr>\n<td>M4A</td>\n<td><a href="/wiki/North_York" title="North York">North York</a></td>\n<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>\n</td></tr>\n<tr>\n<td>M5A</td>\n<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>\n<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>\n</td></tr>\n<tr>\n<td>M6A</td>\n<td><a href="/wiki/North_York" title="North York">North York</a></td>\n<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>\n</td></tr>\n<tr>\

Transform the parse result to an object list(with properties: Postcode, Borough & Neighbourhood)

In [75]:
data_list = []
for idx, tr in enumerate(result.find_all('tr')):
    if idx != 0:
        tds = tr.find_all('td')
        data_list.append({
            'PostalCode': tds[0].contents[0],
            'Borough': tds[1].contents[0],
            'Neighbourhood': tds[2].contents[0],
        })
print(data_list[0:10])

[{'PostalCode': 'M1A', 'Borough': 'Not assigned', 'Neighbourhood': 'Not assigned\n'}, {'PostalCode': 'M2A', 'Borough': 'Not assigned', 'Neighbourhood': 'Not assigned\n'}, {'PostalCode': 'M3A', 'Borough': <a href="/wiki/North_York" title="North York">North York</a>, 'Neighbourhood': <a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>}, {'PostalCode': 'M4A', 'Borough': <a href="/wiki/North_York" title="North York">North York</a>, 'Neighbourhood': <a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>}, {'PostalCode': 'M5A', 'Borough': <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a>, 'Neighbourhood': <a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>}, {'PostalCode': 'M6A', 'Borough': <a href="/wiki/North_York" title="North York">North York</a>, 'Neighbourhood': <a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>}, {'PostalCode': 'M6A', 'Borough': <a href="/wiki/North_York" title="North Y

create the data_frame:

In [65]:
import pandas as pd
data_frame = pd.DataFrame(data_list)

In [66]:
data_frame.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,[North York],[Parkwoods]
3,M4A,[North York],[Victoria Village]
4,M5A,[Downtown Toronto],[Harbourfront]
5,M6A,[North York],[Lawrence Heights]
6,M6A,[North York],[Lawrence Manor]
7,M7A,[Queen's Park],Not assigned\n
8,M8A,Not assigned,Not assigned\n
9,M9A,[Downtown Toronto],[Queen's Park]


Define a function to process the Borough & Neighbourhood data:
    1. clear all the '\n's in the text;
    2. extract title from the <a href structrue;
    3. return the title.

In [85]:
def clean_data(x):
    result = str(x)
    result = result.replace('\n','')
    result = result.replace(',','')
    if('Not assigned' in result):
        return result
    if('href' not in result):
        return result
    #print(x)
    return re.findall('title="(.+?)"', result)[0]

Apply the function to 'Borough' and 'Neighbourhood' columns, and drop all rows where borough is Not assigned:

In [86]:
data_frame['Neighbourhood'] = data_frame['Neighbourhood'].apply(lambda x:clean_data(x))
data_frame['Borough'] = data_frame['Borough'].apply(lambda x:clean_data(x))

data_frame = data_frame[data_frame['Borough']!='Not assigned'].reset_index(drop=True)

replace Not-assigned Neighbourhood with the value of Borough

In [76]:
for index, row in data_frame.iterrows():
    if(row['Neighbourhood'] == 'Not assigned'):
        row['Neighbourhood'] = row['Borough']

In [87]:
data_frame.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park (Toronto),Queen's Park (Toronto)
6,M9A,Downtown Toronto,Queen's Park (Toronto)
7,M1B,Scarborough Toronto,Rouge Toronto
8,M1B,Scarborough Toronto,Malvern Toronto
9,M3B,North York,Don Mills North


Finally we get the postalcode data:

In [78]:
data_frame.shape

(210, 3)

Group the data by ['PostalCode','Borough'] and join the Beighbourhood with comma.

In [89]:
data_frame = data_frame.groupby(['PostalCode','Borough'])['Neighbourhood'].agg(lambda x:','.join(x)).reset_index()

In [92]:
data_frame.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough Toronto,"Rouge Toronto,Malvern Toronto"
1,M1C,Scarborough Toronto,"Highland Creek (Toronto),Rouge Hill,Port Union..."
2,M1E,Scarborough Toronto,"Guildwood,Morningside Toronto,West Hill Toronto"
3,M1G,Scarborough Toronto,Woburn Toronto
4,M1H,Scarborough Toronto,Cedarbrae


In [90]:
data_frame.shape

(103, 3)

In [93]:
data_frame[data_frame['PostalCode']=='M5G']

,PostalCode,Borough,Neighbourhood
57,M5G,Downtown Toronto,Central Bay Street


Get the geospatial_coordinates data from the csv file:

In [96]:
geospatial_coordinates = pd.read_csv("./Geospatial_Coordinates.csv")

In [97]:
geospatial_coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Join dataframe according to the PostalCode:

In [99]:
result_data = pd.merge(left=data_frame, right=geospatial_coordinates, how='left', left_on='PostalCode', right_on='Postal Code')


In [101]:
result_data = result_data.drop(['Postal Code'], axis = 1)

The result is :

In [102]:
result_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough Toronto,"Rouge Toronto,Malvern Toronto",43.806686,-79.194353
1,M1C,Scarborough Toronto,"Highland Creek (Toronto),Rouge Hill,Port Union...",43.784535,-79.160497
2,M1E,Scarborough Toronto,"Guildwood,Morningside Toronto,West Hill Toronto",43.763573,-79.188711
3,M1G,Scarborough Toronto,Woburn Toronto,43.770992,-79.216917
4,M1H,Scarborough Toronto,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York Toronto,Weston Toronto,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Mou...",43.739416,-79.588437


In [103]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [104]:
neighborhoods = result_data[['Borough','Neighbourhood','Latitude','Longitude']]

In [105]:
neighborhoods

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough Toronto,"Rouge Toronto,Malvern Toronto",43.806686,-79.194353
1,Scarborough Toronto,"Highland Creek (Toronto),Rouge Hill,Port Union...",43.784535,-79.160497
2,Scarborough Toronto,"Guildwood,Morningside Toronto,West Hill Toronto",43.763573,-79.188711
3,Scarborough Toronto,Woburn Toronto,43.770992,-79.216917
4,Scarborough Toronto,Cedarbrae,43.773136,-79.239476
5,Scarborough Toronto,Scarborough Village,43.744734,-79.239476
6,Scarborough Toronto,"East Birchmount Park,Ionview,Kennedy Park Toronto",43.727929,-79.262029
7,Scarborough Toronto,"Clairlea,Golden Mile Toronto,Oakridge Toronto",43.711112,-79.284577
8,Scarborough Toronto,"Cliffcrest,Cliffside Toronto,Scarborough Villa...",43.716316,-79.239476
9,Scarborough Toronto,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [106]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 12 boroughs and 103 neighborhoods.


In [107]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tnt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [109]:
# create map of Toronto using latitude and longitude values
map_all = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_all

In [111]:
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough Toronto,"Rouge Toronto,Malvern Toronto",43.806686,-79.194353
1,Scarborough Toronto,"Highland Creek (Toronto),Rouge Hill,Port Union...",43.784535,-79.160497
2,Scarborough Toronto,"Guildwood,Morningside Toronto,West Hill Toronto",43.763573,-79.188711
3,Scarborough Toronto,Woburn Toronto,43.770992,-79.216917
4,Scarborough Toronto,Cedarbrae,43.773136,-79.239476


In [112]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [129]:
kclusters = 3
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_data[['Latitude','Longitude']])

In [130]:
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2], dtype=int32)

In [131]:
toronto_data['Cluster Labels'] = kmeans.labels_

In [132]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9.5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood'], toronto_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters